<a href="https://colab.research.google.com/github/Soma-codeAI/Google_ADK_Examples/blob/main/notebooks/FundInsights_Investment_ResearchAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install google-generativeai
%pip install google-adk
%pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import auth

auth.authenticate_user(project_id="<Project_ID>")

In [2]:
import vertexai
from vertexai import agent_engines

vertexai.init(
    project="<Project_ID>",
    location="us-east4",
    staging_bucket="<Gcloud Bucket Name>",
)

In [3]:
# Set up guardrails for the model
from google.genai import types

safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.OFF,
    ),
]

In [4]:
# Content Generation parameters, This is where you get your model to be creative
from google.genai import types


generate_content_config = types.GenerateContentConfig(
   safety_settings=safety_settings,
   temperature=0.28,
   max_output_tokens=1000,
   top_p=0.95,
)

In [7]:
project_id="crested-aquifer-456623-p3"
location="us-east4"

from google.adk.agents import Agent, SequentialAgent
from vertexai.preview.reasoning_engines import AdkApp
import vertexai
from vertexai import agent_engines
from google.genai import types
import requests
from bs4 import BeautifulSoup
import pandas as pd


model1 = vertexai.generative_models.GenerativeModel(model_name="gemini-2.0-flash")


# Ticket Extracter Writer Agent
# Takes the initial specification (from user query) and writes code.
ticker_extract_agent = Agent(
    name="TicketExtracterAgent",
    model="gemini-2.0-flash",
    # Change 3: Improved instruction
    instruction="Extract the fund name and ticker symbol from the provided text. Output in the format: 'Fund Name: [Fund Name], Ticker: [Ticker Symbol]'.",
    description="Extract the fund ticker symbol from the user's message.",
    output_key="ticker_symbol" # Stores output in state['generated_code']
)


# prompt: Write a function to return the url for the given ticket symbol.     Example1: Soma Fund, Ticker: SOMX
#     https://www.dataroma.com/m/holdings.php?m=SOMX
#     Example2: Royce Fund, Ticker: VALX
#     https://www.dataroma.com/m/holdings.php?m=VALX

def get_dataroma_url(ticker_symbol: str) -> str:
  """
  Generates the Dataroma URL for a given ticket symbol.

  Args:
      ticker_symbol (str): The stock ticker symbol (e.g., "SOMAX", "VALX").

  Returns:
      str: The Dataroma URL for the ticker symbol.
  """
  base_url = "https://www.dataroma.com/m/holdings.php?m="
  return f"{base_url}{ticker_symbol}"


def get_fund_holdings(url: str) -> str: # Changed return type hint to str
    """
    Scrapes the fund holdings from a given Dataroma URL.

    Args:
        url (str): The Dataroma URL for the fund holdings.

    Returns:
        str: A prompt to analyze the fund holdings, or an error message.
    """

    try:
        request = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Company info@company.com)"})
        request.raise_for_status()  # Raise an exception for bad status codes
        text_to_summarize = request.text

        prompt = f"""
        You are a highly skilled financial analyst specializing in analyzing fund holdings (13F filings) to identify potential investment ideas and assess risks. Your analysis should be data-driven, objective, and focused on providing actionable insights.

        Follow these steps to analyze fund holdings and generate investment ideas and risk assessments:

1.  **Data Extraction:**
    *   Identify and extract relevant data from the 13F filings, including:
        *   List of holdings
        *   Number of shares held
        *   Market value of holdings
        *   Changes in holdings (new positions, increases, decreases, sold out positions)
2.  **Holdings Analysis:**
    *   Analyze the extracted holdings data to identify:
        *   Top holdings by market value
        *   Sectors with the highest concentration of investments
        *   Stocks with the most significant changes in ownership
3.  **Investment Idea Generation:**
    *   Based on the holdings analysis, generate potential investment ideas, considering:
        *   Stocks that are widely held by successful fund managers
        *   Stocks with increasing ownership, indicating positive sentiment
        *   Undervalued stocks relative to their intrinsic value
4.  **Risk Assessment:**
    *   Assess potential risks associated with the identified investment ideas, including:
        *   Concentration risk (over-reliance on a few key holdings or sectors)
        *   Liquidity risk (difficulty in selling large positions without affecting the market price)
        *   Market risk (sensitivity to overall market movements)
5.  **Report Generation:**
    *   Present your findings in a clear and concise report, including:
        *   Executive summary of key investment ideas and risks
        *   Detailed analysis of fund holdings
        *   Supporting data and charts
        *   Recommendations for further research and due diligence

Ensure your analysis is objective and data-driven, relying on the information available in the 13F filings and other credible sources.
        {text_to_summarize}
        """

        #response = gemini_model.generate_content(prompt)
        return prompt # Return the prompt string

    except requests.exceptions.RequestException as e:
        return f"Error fetching the webpage: {e}"
    except Exception as e:
        return f"An error occurred while parsing the page: {e}"


fund_url_and_holdings_agent = Agent(
    name='fund_url_and_holdings_agent',
    model = "gemini-2.0-flash",                 # Required.
    #generate_content_config=generate_content_config,  # Optional.
    instruction= 'You are an agent who receives a fund ticker symbol from the previous agent in the pipeline (available as the \'ticker_symbol\' output from the previous step). First, call the `get_dataroma_url` tool with the ticker symbol to get the Dataroma URL. Then, call the `get_fund_holdings` tool with the obtained URL and return the generated prompt.',
    description='This agent gets the Dataroma URL for a fund ticker and then scrapes the fund holdings data from the URL to generate an analysis prompt.',
    tools=[get_dataroma_url, get_fund_holdings], # Include both functions as tools
    output_key="analysis_prompt"
    )


# Report Generator Agent
# Takes the analysis prompt and generates an investment research report.
report_generator_agent = Agent(
    name="ReportGeneratorAgent",
    model="gemini-2.0-flash", # Or another suitable model
    instruction="You are a financial analyst. Generate an investment research report based on the analysis prompt provided. Structure the report clearly, including an executive summary, analysis of holdings, investment ideas, and risk assessment, following the guidance in the prompt.",
    description="Generates an investment research report from a provided analysis prompt analysis_prompt.",
    output_key="investment_report" # Stores the generated report
)

# --- 2. Create the SequentialAgent ---
# This agent orchestrates the pipeline by running the sub_agents in order.
code_pipeline_agent = SequentialAgent(
    name="FundInsightsAgent",
    sub_agents=[ticker_extract_agent, fund_url_and_holdings_agent, report_generator_agent],
    description="Executes a sequence of ticket extractror and fund_url_and_holdings_agent",
    # The agents will run in the order provided: Writer -> Reviewer -> Refactorer
)


# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = code_pipeline_agent

app = AdkApp(agent=root_agent)

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [8]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [9]:
app.list_sessions(user_id="u_123")
session = app.create_session(user_id="u_123")

for event in app.stream_query(
      user_id="u_123",
      session_id=session.id,
      message="infund url for the ticker FE",
    ):
      print(event)

print(event['content']['parts'][0]['text'])

{'content': {'parts': [{'text': 'Fund Name: inFUND, Ticker: FE\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 11, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 11}], 'prompt_token_count': 75, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 75}], 'total_token_count': 86, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-ab731afa-c0ac-4fbd-b225-3d98b63ca3e4', 'author': 'TicketExtracterAgent', 'actions': {'state_delta': {'ticker_symbol': 'Fund Name: inFUND, Ticker: FE\n'}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'GDF2VJCc', 'timestamp': 1751847706.988584}


{'content': {'parts': [{'function_call': {'id': 'adk-55cd676b-a450-4a6f-a4cc-8d38289d7077', 'args': {'ticker_symbol': 'FE'}, 'name': 'get_dataroma_url'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 10, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 10}], 'prompt_token_count': 306, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 306}], 'total_token_count': 316, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-ab731afa-c0ac-4fbd-b225-3d98b63ca3e4', 'author': 'fund_url_and_holdings_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'bIc5pz9I', 'timestamp': 1751847708.492188}
{'content': {'parts': [{'function_response': {'id': 'adk-55cd676b-a450-4a6f-a4cc-8d38289d7077', 'name': 'get_dataroma_url', 'response': {'result': 'https://www.dataroma.com/m/holdings.php?m=FE'}}}], 'role': 'user'}, 'i

{'content': {'parts': [{'function_call': {'id': 'adk-06cdf732-1f21-464d-83c9-3814907f9dff', 'args': {'url': 'https://www.dataroma.com/m/holdings.php?m=FE'}, 'name': 'get_fund_holdings'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 24, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 24}], 'prompt_token_count': 341, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 341}], 'total_token_count': 365, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-ab731afa-c0ac-4fbd-b225-3d98b63ca3e4', 'author': 'fund_url_and_holdings_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'MEdhSWAn', 'timestamp': 1751847709.822606}
{'content': {'parts': [{'function_response': {'id': 'adk-06cdf732-1f21-464d-83c9-3814907f9dff', 'name': 'get_fund_holdings', 'response': {'result': '\n        You are a highly skilled f

In [13]:
import json
from IPython.display import HTML

# Extract the JSON string from the event
json_string_wrapped = event['content']['parts'][0]['text']

# Remove the markdown code block wrapping
# Assumes the JSON is within ```json ... ```
import re
match = re.search(r'```json\n(.*)\n```', json_string_wrapped, re.DOTALL)
if match:
    json_string = match.group(1)
else:
    json_string = json_string_wrapped # Fallback if not wrapped as expected

# Parse the JSON string
data = json.loads(json_string)

# Extract the investment report data
report = data.get('investment_report', {})

# Function to flatten holdings data for the table
def flatten_holdings(holdings_list):
    flattened = []
    for holding in holdings_list:
        flattened.append({
            'Ticker': holding.get('ticker', ''),
            'Name': holding.get('name', ''),
            '% of Portfolio': holding.get('percentage_of_portfolio', ''),
            'Market Value': holding.get('market_value', '')
        })
    return flattened

# Function to flatten significant changes data
def flatten_changes(changes_dict, change_type):
    flattened = []
    changes = changes_dict.get(change_type, [])
    for change in changes:
        flattened.append({
            'Ticker': change.get('ticker', ''),
            'Name': change.get('name', ''),
            'Change': change.get('change', '')
        })
    return flattened

# Prepare data for tables
top_holdings_data = flatten_holdings(report.get('holdings_analysis', {}).get('top_holdings', []))
increase_changes_data = flatten_changes(report.get('holdings_analysis', {}).get('significant_changes', {}), 'increases')
decrease_changes_data = flatten_changes(report.get('holdings_analysis', {}).get('significant_changes', {}), 'decreases')
investment_ideas_data = report.get('investment_ideas', [])
risk_assessment_data = report.get('risk_assessment', {})
recommendations_data = report.get('recommendations', [])


# Function to generate HTML table from data
def generate_html_table(data, title):
    if not data:
        return f"<h2>{title}</h2><p>No data available.</p>"

    html = f"<h2>{title}</h2>"
    html += "<table border='1'>"
    # Add table headers
    html += "<tr>"
    if isinstance(data, list) and data:
        if isinstance(data[0], dict):
            for key in data[0].keys():
                html += f"<th>{key}</th>"
        elif isinstance(data[0], str):
             html += "<th>Recommendation</th>"
    html += "</tr>"
    # Add table rows
    for row in data:
        html += "<tr>"
        if isinstance(row, dict):
            for value in row.values():
                html += f"<td>{value}</td>"
        elif isinstance(row, str):
            html += f"<td>{row}</td>"
        html += "</tr>"
    html += "</table>"
    return html

# Generate HTML for each section
top_holdings_html = generate_html_table(top_holdings_data, "Top Holdings")
increases_html = generate_html_table(increase_changes_data, "Significant Increases in Holdings")
decreases_html = generate_html_table(decrease_changes_data, "Significant Decreases in Holdings")

# Format Investment Ideas
investment_ideas_html = "<h2>Investment Ideas</h2>"
if investment_ideas_data:
    investment_ideas_html += "<ul>"
    for idea in investment_ideas_data:
        investment_ideas_html += f"<li><b>{idea.get('ticker', '')}:</b> {idea.get('rationale', '')}</li>"
    investment_ideas_html += "</ul>"
else:
    investment_ideas_html += "<p>No investment ideas available.</p>"

# Format Risk Assessment
risk_assessment_html = "<h2>Risk Assessment</h2>"
if risk_assessment_data:
    risk_assessment_html += "<ul>"
    for key, value in risk_assessment_data.items():
        risk_assessment_html += f"<li><b>{key.replace('_', ' ').title()}:</b> {value}</li>"
    risk_assessment_html += "</ul>"
else:
    risk_assessment_html += "<p>No risk assessment available.</p>"

# Format Recommendations
recommendations_html = generate_html_table(recommendations_data, "Recommendations")


# Combine all HTML parts
full_html_output = f"""
<h1>Investment Research Report</h1>
{generate_html_table([{'Executive Summary': report.get('executive_summary', 'N/A')}], "Executive Summary")}
{top_holdings_html}
{increases_html}
{decreases_html}
{investment_ideas_html}
{risk_assessment_html}
{recommendations_html}
"""

# Display the HTML
display(HTML(full_html_output))